In [10]:
% load_ext autoreload
% autoreload 2
# Plot configurations
% matplotlib inline
from __future__ import print_function
import tensorflow as tf
import numpy as np
from cifar_utils import load_data
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Load the raw CIFAR-10 data.
X_train, y_train = load_data(mode='train')

mask = np.arange(X_train.shape[0])
np.random.shuffle(mask)
X_train = X_train[mask]
y_train = y_train[mask]

./data/cifar-10-python.tar.gz already exists. Begin extracting...


In [16]:
################## global contrast normalization ######################
lamda = 10
epsilon = 1e-7

#Because of memory issue, the pre-process of test data has to be after training data
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
contrast = np.sqrt(lamda + np.mean(X_train**2,axis=0))
X_train = X_train / np.maximum(contrast, epsilon)

################## ZCA whitening #########################
temp = []
principal_components = []
for c in range(3):
    X = X_train[:,c*1024:(c+1)*1024]
    cov = np.dot(X.T, X) / (X.shape[0]-1)
    u, s, _ = np.linalg.svd(cov)
    principal_components.append( np.dot(np.dot(u, np.diag(1. / np.sqrt(s + 10e-7))), u.T) )

    # Apply ZCA whitening
    whitex = np.dot(X, principal_components[c])
    temp.append(whitex)

X_train = np.append(temp[0],temp[1],axis=1)
X_train = np.append(X_train,temp[2],axis=1)
X_train = X_train.reshape([50000,3,32,32]).transpose((0,2,3,1))

In [17]:
num_training = 45000
num_validation = 5000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

X_test = []
y_test = []

X_test, y_test = load_data(mode='test')
X_test = X_test.astype(np.float32) - mean_image
contrast = np.sqrt(lamda + np.mean(X_test**2,axis=0))
X_test = X_test / np.maximum(contrast, epsilon)

for c in range(3): 
    X = X_test[:,c*1024:(c+1)*1024]
    whitex = np.dot(X, principal_components[c])
    temp[c] = whitex

X_test = np.append(temp[0],temp[1],axis=1)
X_test = np.append(X_test,temp[2],axis=1)
X_test = X_test.reshape([10000,3,32,32]).transpose((0,2,3,1))

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
#print('Test data shape: ', X_test.shape)
#print('Test labels shape: ', y_test.shape)

./data/cifar-10-python.tar.gz already exists. Begin extracting...


Train data shape:  (45000, 32, 32, 3)
Train labels shape:  (45000,)
Validation data shape:  (5000, 32, 32, 3)
Validation labels shape:  (5000,)


In [18]:
del temp
del whitex
del principal_components
del u
del s
del X

In [7]:
# No Regularizer
from binary import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, X_test, y_test,
                     is_binary = False, 
                     is_stochastic = False, 
                     conv_featmap = [128, 128, 256, 256, 512, 512],
                     fc_units = [1024, 1024],
                     conv_kernel_size = [3, 3, 3, 3, 3, 3],
         pooling_size = [2, 2, 2],
         lr_start = 0.001,
         lr_end = 0.0001, 
         epoch = 20,
         batch_size = 50,
         is_drop_out = False,
         verbose = True,
         pre_trained_model = None)

TypeError: Incompatible return types of true_fn and false_fn: The two structures don't have the same nested structure.

First structure: type=tuple str=([<tf.Tensor 'Adam_optimize/cond/truediv:0' shape=(3, 3, 3, 128) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_1:0' shape=(3, 3, 128, 128) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_2:0' shape=(3, 3, 128, 256) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_3:0' shape=(3, 3, 256, 256) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_4:0' shape=(3, 3, 256, 512) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_5:0' shape=(3, 3, 512, 512) dtype=float32>], [<tf.Tensor 'Adam_optimize/cond/truediv_6:0' shape=(128,) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_7:0' shape=(128,) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_8:0' shape=(256,) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_9:0' shape=(256,) dtype=float32>, <tf.Tensor 'Adam_optimize/cond/truediv_10:0' shape=(512,) 

In [19]:
# BinaryConnect det.
from binary import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, X_test, y_test,
         is_binary = True, 
         is_stochastic = False, 
         conv_featmap = [128, 128, 256, 256, 512, 512],
         fc_units = [1024, 1024],
         conv_kernel_size = [3, 3, 3, 3, 3, 3],
         pooling_size = [2, 2, 2],
         lr_start = 0.01,
         lr_end = 0.001, 
         epoch = 200,
         batch_size = 50,
         is_drop_out = False,
         verbose = True,
         pre_trained_model = None,
         record_tensorboard=False)

number of batches for training: 900


epoch 1 


average train loss: 0.6243072298169136 ,  average accuracy : 49.14%


validation accuracy : 51.12%
* Best accuracy: 51.12%


epoch 2 


...

average train loss: 0.3857903302046988 ,  average accuracy : 96.18666666666667%


validation accuracy : 80.9%
epoch 16 


average train loss: 0.3849865636560652 ,  average accuracy : 96.3711111111111%


validation accuracy : 80.12%
epoch 17 


KeyboardInterrupt: 

In [9]:
# BinaryConnect stoch.
from binary import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, X_test, y_test,
         is_binary = True, 
         is_stochastic = True, 
         conv_featmap = [128, 128, 256, 256, 512, 512],
         fc_units = [1024, 1024],
         conv_kernel_size = [3, 3, 3, 3, 3, 3],
         pooling_size = [2, 2, 2],
         lr_start = 0.1,
         lr_end = 0.0001, 
         epoch = 50,
         batch_size = 50,
         is_drop_out = False,
         verbose = True,
         pre_trained_model = None)

NameError: name 'X_val' is not defined